<a href="https://colab.research.google.com/github/jipearson/blue-team/blob/merging_datasets/db_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Spark

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad

In [2]:
# Download Postgres driver
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-05-16 05:17:12--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.8’

postgresql-42.2.16. 100%[===================>] 979.38K  6.06MB/s    in 0.2s    

2021-05-16 05:17:12 (6.06 MB/s) - ‘postgresql-42.2.16.jar.8’ saved [1002883/1002883]



In [3]:
# Create Spark session and add driver to Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

# ETL Voting Data

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
import pandas as pd
import numpy as np
# Read in 2000-2019 census data
url ="https://migration-election-impact.s3.us-east-2.amazonaws.com/Resources/Census_00_19.csv"
spark.sparkContext.addFile(url)
census_df = spark.read.csv(SparkFiles.get("Census_00_19.csv"), sep=",", header=True, inferSchema=True)
census_df = census_df.select("*").toPandas()

In [5]:
# Read in 1996-2020 presidential election data
url ="https://migration-election-impact.s3.us-east-2.amazonaws.com/Resources/Presidential_96_2020.csv"
spark.sparkContext.addFile(url)
presidential_df = spark.read.csv(SparkFiles.get("Presidential_96_2020.csv"), sep=",", header=True, inferSchema=True)
presidential_df = presidential_df.select("*").toPandas()

In [6]:
# Read in past 5 voting patterns data
url = "https://migration-election-impact.s3.us-east-2.amazonaws.com/Resources/PAST_5_ELECTION_VOTING.csv"
spark.sparkContext.addFile(url)
five_election_df = spark.read.csv(SparkFiles.get("PAST_5_ELECTION_VOTING.csv"), sep=",", header=True, inferSchema=True)
five_election_df = five_election_df.select("*").toPandas()

In [7]:
# Clean presidential_df
presidential_df['Outcome'] = np.where(presidential_df.Winning_Party == "REPUBLICAN", 1, 0)
presidential_df = presidential_df[presidential_df.year >= 2000]
presidential_df = presidential_df.rename(columns=str.upper)
presidential_df = presidential_df[['YEAR','STATE','OUTCOME']]

In [8]:
# Clean Census_df
census_df.STATE_NAME = census_df.STATE_NAME.str.upper()
census_df["PCT_WHITE"] = census_df["RACE_WHITE"] / census_df["TOTAL_POPULATION"]
census_df["PCT_BLACK"] = census_df["RACE_BLACK"] / census_df["TOTAL_POPULATION"]
census_df["PCT_HISPANIC"] = census_df["ORIGIN_HISPANIC"] / census_df["TOTAL_POPULATION"]
census_df["PCT_OVER_60"] = (census_df["AGE_60_TO_79"] + census_df["AGE_80_AND_OVER"]) /census_df["TOTAL_POPULATION"]
census_df["PCT_UNDER_40"] = (census_df["AGE_0_TO_19"] + census_df["AGE_20_TO_39"]) /census_df["TOTAL_POPULATION"]
census_df.loc[census_df["YEAR"] == 2019, "YEAR"] = 2020
census_df = census_df[['YEAR', 'STATE_NAME', 'REGION', 'PCT_WHITE', 'PCT_BLACK', 'PCT_HISPANIC', 'PCT_OVER_60', 'PCT_UNDER_40' ]]

In [9]:
census_df = pd.get_dummies(census_df, columns=["REGION"])

In [10]:
election_merged_df = pd.merge(presidential_df, census_df, how="inner", left_on=["YEAR", "STATE"], right_on = ["YEAR", "STATE_NAME"])
election_merged_df

,YEAR,STATE,OUTCOME,STATE_NAME,PCT_WHITE,PCT_BLACK,PCT_HISPANIC,PCT_OVER_60,PCT_UNDER_40,REGION_Midwest,REGION_Northeast,REGION_South,REGION_West
0,2000,ALABAMA,1,ALABAMA,0.718555,0.260610,0.017049,0.173121,0.563600,0,0,1,0
1,2000,ALASKA,1,ALASKA,0.712087,0.036541,0.041236,0.084585,0.627195,0,0,0,1
2,2000,ARIZONA,1,ARIZONA,0.880687,0.032933,0.252545,0.169827,0.587959,0,0,0,1
3,2000,ARKANSAS,1,ARKANSAS,0.816632,0.157303,0.032493,0.183815,0.559322,0,0,1,0
4,2000,CALIFORNIA,0,CALIFORNIA,0.778388,0.070299,0.323768,0.140014,0.609940,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,2020,VIRGINIA,0,VIRGINIA,0.693883,0.198806,0.097759,0.221305,0.519141,0,0,1,0
302,2020,WASHINGTON,0,WASHINGTON,0.785001,0.043589,0.130234,0.220739,0.531032,0,0,0,1
303,2020,WEST VIRGINIA,1,WEST VIRGINIA,0.934701,0.036052,0.017388,0.277159,0.465235,0,0,1,0
304,2020,WISCONSIN,0,WISCONSIN,0.870389,0.067076,0.070968,0.243544,0.502778,1,0,0,0


In [11]:
# Clean Past 5 Election Data
five_election_df = five_election_df[five_election_df["YEAR"] >= 2000]
five_election_df = five_election_df[['YEAR', 'STATE', 'PAST_5_ELECTION_VOTING']]
five_election_df

,YEAR,STATE,PAST_5_ELECTION_VOTING
6,2000,ALABAMA,1.0
7,2004,ALABAMA,1.0
8,2008,ALABAMA,1.0
9,2012,ALABAMA,1.0
10,2016,ALABAMA,1.0
...,...,...,...
607,2004,WYOMING,1.0
608,2008,WYOMING,1.0
609,2012,WYOMING,1.0
610,2016,WYOMING,1.0


In [12]:
election_merged_df = pd.merge(election_merged_df, five_election_df, how='inner', on=['YEAR','STATE'])
election_merged_df.drop(['STATE_NAME'], axis=1, inplace=True)

In [13]:
# Convert Pandas DF to Spark DF
election_merged_df = spark.createDataFrame(election_merged_df)
election_merged_df

DataFrame[YEAR: bigint, STATE: string, OUTCOME: bigint, PCT_WHITE: double, PCT_BLACK: double, PCT_HISPANIC: double, PCT_OVER_60: double, PCT_UNDER_40: double, REGION_Midwest: bigint, REGION_Northeast: bigint, REGION_South: bigint, REGION_West: bigint, PAST_5_ELECTION_VOTING: double]

# Load election data into RDS

In [14]:
from google.colab import files
uploaded = files.upload()

Saving config.py to config (4).py


In [15]:
from config import password 
# Configure settings for RDS
mode = "overwrite"
jdbc_url="jdbc:postgresql://presidential-migration-db.cexjpewgnljl.us-east-2.rds.amazonaws.com:5432/presidential_migration_db"
config = {"user":"postgres_blue",
          "password": f'{password}',
          "driver":"org.postgresql.Driver"}

In [16]:
# Load cleaned election results into RDS
election_merged_df.write.jdbc(url=jdbc_url, table='election', mode=mode, properties=config)

# ETL Demographic Data

## Degrees_df

In [19]:
# Read in cleaned college education data
url = "https://migration-election-impact.s3.us-east-2.amazonaws.com/Resources/clean_degrees_by_state.csv"
spark.sparkContext.addFile(url)
degrees_df = spark.read.csv(SparkFiles.get("clean_degrees_by_state.csv"), sep=",", header=True, inferSchema=True )
degrees_df = degrees_df.select("*").toPandas()

In [20]:
# Change column headings to upper case and convert STATE column values to upper case
degrees_df = degrees_df.rename(columns=str.upper)
degrees_df['STATE'] = degrees_df['STATE'].str.upper()

In [21]:
degrees_2000 = degrees_df[(degrees_df.YEAR ==2005)]
degrees_2004 = degrees_df[(degrees_df.YEAR ==2005)]
degrees_2020 = degrees_df[(degrees_df.YEAR ==2019)]
degrees_2000.loc[degrees_2000['YEAR'] == 2005, 'YEAR'] = 2000
degrees_2004.loc[degrees_2004['YEAR'] == 2005, 'YEAR'] = 2004
degrees_2020.loc[degrees_2020['YEAR'] == 2019, 'YEAR'] = 2020

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [22]:
degrees_df = degrees_df.append(degrees_2000,ignore_index=True)
degrees_df = degrees_df.append(degrees_2004,ignore_index=True)
degrees_df = degrees_df.append(degrees_2020,ignore_index=True)

In [23]:
# Filter df to include only presidential election year data. 
degrees_df = degrees_df[degrees_df.YEAR.isin(["2000","2004","2008","2012","2016","2020"])].sort_values(by=['YEAR']).rename(columns = {'#_BAS': 'BACHELORS_DEGREE_TOT'}, inplace = False)
degrees_df

,STATE,YEAR,BACHELORS_DEGREE_TOT
815,WYOMING,2000,29830
786,MASSACHUSETTS,2000,780520
785,MARYLAND,2000,582280
784,MAINE,2000,85985
783,LOUISIANA,2000,267430
...,...,...,...
887,MARYLAND,2020,712675
888,MASSACHUSETTS,2020,974100
889,MICHIGAN,2020,851380
891,MISSISSIPPI,2020,172285


## Incarceration_df

In [25]:
# Read in incarceration data
url ="https://migration-election-impact.s3.us-east-2.amazonaws.com/Resources/clean_incarceration.csv"
spark.sparkContext.addFile(url)
incarceration_df = spark.read.csv(SparkFiles.get("clean_incarceration.csv"), sep=",", header=True, inferSchema=True)
incarceration_df = incarceration_df.select("*").toPandas()

In [26]:
# Change column headings to upper case and convert STATE column values to upper case
incarceration_df = incarceration_df.rename(columns=str.upper)
incarceration_df['STATE'] = incarceration_df['STATE'].str.upper()
incarceration_df

,YEAR,STATE,PRISON_JAIL_RATE,VALUE
0,2000,ALABAMA,Jail Rate,203.1
1,2000,ALABAMA,Prison Rate,584.8
2,2000,ALASKA,Prison Rate,339.1
3,2000,ARIZONA,Jail Rate,196.9
4,2000,ARIZONA,Prison Rate,491.8
...,...,...,...,...
1441,2015,VIRGINIA,Prison Rate,456.7
1442,2015,WASHINGTON,Prison Rate,252.2
1443,2015,WEST VIRGINIA,Prison Rate,386.0
1444,2015,WISCONSIN,Prison Rate,376.6


In [27]:
#Split prison and jail rates into separate columns and 
jail_df = incarceration_df[(incarceration_df.PRISON_JAIL_RATE == 'Jail Rate')].rename(columns = {'VALUE': 'JAIL_RATE'}, inplace = False).drop(columns=['PRISON_JAIL_RATE'])
prison_df = incarceration_df[(incarceration_df.PRISON_JAIL_RATE == 'Prison Rate')].rename(columns = {'VALUE': 'PRISON_RATE'}, inplace = False).drop(columns=['PRISON_JAIL_RATE'])
incarceration_df = pd.merge(jail_df, prison_df, how='right', left_on=['YEAR', 'STATE'], right_on = ['YEAR', 'STATE'])
incarceration_df

,YEAR,STATE,JAIL_RATE,PRISON_RATE
0,2000,ALABAMA,203.1,584.8
1,2000,ALASKA,NaN,339.1
2,2000,ARIZONA,196.9,491.8
3,2000,ARKANSAS,163.1,442.5
4,2000,CALIFORNIA,193.1,471.9
...,...,...,...,...
811,2015,VIRGINIA,NaN,456.7
812,2015,WASHINGTON,NaN,252.2
813,2015,WEST VIRGINIA,NaN,386.0
814,2015,WISCONSIN,NaN,376.6


In [28]:
# Duplicate 2015 values for 2016/2020 and append to df
incarceration_2016 = incarceration_df[(incarceration_df.YEAR ==2015)]
incarceration_2020 = incarceration_df[(incarceration_df.YEAR ==2015)]

incarceration_2016.loc[incarceration_2016['YEAR'] == 2015, 'YEAR'] = 2016
incarceration_2020.loc[incarceration_2020['YEAR'] == 2015, 'YEAR'] = 2020

incarceration_df = incarceration_df.append(incarceration_2016,ignore_index=True)
incarceration_df = incarceration_df.append(incarceration_2020,ignore_index=True)
incarceration_df

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,YEAR,STATE,JAIL_RATE,PRISON_RATE
0,2000,ALABAMA,203.1,584.8
1,2000,ALASKA,NaN,339.1
2,2000,ARIZONA,196.9,491.8
3,2000,ARKANSAS,163.1,442.5
4,2000,CALIFORNIA,193.1,471.9
...,...,...,...,...
913,2020,VIRGINIA,NaN,456.7
914,2020,WASHINGTON,NaN,252.2
915,2020,WEST VIRGINIA,NaN,386.0
916,2020,WISCONSIN,NaN,376.6


In [29]:
# Filter df to include only presidential election year data. 
incarceration_df = incarceration_df[incarceration_df.YEAR.isin(["2000","2004","2008","2012","2016","2020"])].sort_values(by=['YEAR'])
incarceration_df

,YEAR,STATE,JAIL_RATE,PRISON_RATE
0,2000,ALABAMA,203.1,584.8
28,2000,NEVADA,219.7,498.6
29,2000,NEW HAMPSHIRE,47.3,182.0
30,2000,NEW JERSEY,128.8,353.3
31,2000,NEW MEXICO,275.4,256.3
...,...,...,...,...
887,2020,MARYLAND,NaN,338.8
888,2020,MASSACHUSETTS,NaN,179.3
889,2020,MICHIGAN,NaN,429.2
891,2020,MISSISSIPPI,NaN,608.8


## Income_df

In [32]:
# Read in incarceration data
url ="https://migration-election-impact.s3.us-east-2.amazonaws.com/Resources/clean_med_income.csv"
spark.sparkContext.addFile(url)
income_df = spark.read.csv(SparkFiles.get("clean_med_income.csv"), sep=",", header=True, inferSchema=True)
income_df = income_df.select("*").toPandas()

In [33]:
# Change column headings to upper case and convert STATE column values to upper case
income_df = income_df.rename(columns=str.upper)
income_df['STATE'] = income_df['STATE'].str.upper()

In [34]:
# Convert YEAR to int 
income_df['YEAR'] = income_df['YEAR'].astype(int)
income_df

,YEAR,STATE,MED_HOUSE_INC
0,1984,ALABAMA,17310
1,1984,ALASKA,18333
2,1984,ARIZONA,19132
3,1984,ARKANSAS,19734
4,1984,CALIFORNIA,19948
...,...,...,...
1933,2019,VIRGINIA,57829
1934,2019,WASHINGTON,59536
1935,2019,WEST VIRGINIA,57837
1936,2019,WISCONSIN,62539


In [35]:
# Change 2019 to 2020 so that it can be joined with 2020 election data. 
income_df.loc[income_df['YEAR'] == 2019, 'YEAR'] = 2020

# Filter df to include only presidential election year data. 
income_df = income_df[income_df.YEAR.isin(["2000","2004","2008","2012","2016","2020"])].sort_values(by=['YEAR'])
income_df

,YEAR,STATE,MED_HOUSE_INC
816,2000,ALABAMA,49855
844,2000,NEVADA,32267
845,2000,NEW HAMPSHIRE,32662
846,2000,NEW JERSEY,35284
847,2000,NEW MEXICO,36426
...,...,...,...
1907,2020,MARYLAND,29050
1908,2020,MASSACHUSETTS,30209
1909,2020,MICHIGAN,29442
1911,2020,MISSISSIPPI,31529


## Crimes_df

In [36]:
# Read in Violent Crime data
url ="https://migration-election-impact.s3.us-east-2.amazonaws.com/Resources/clean_violent_crimes.csv"
spark.sparkContext.addFile(url)
crimes_df = spark.read.csv(SparkFiles.get("clean_violent_crimes.csv"), sep=",", header=True, inferSchema=True)
crimes_df = crimes_df.select("*").toPandas()

In [37]:
# Change column headings to upper case and convert STATE column values to upper case
crimes_df = crimes_df.rename(columns=str.upper)
crimes_df['STATE'] = crimes_df['STATE'].str.upper()

In [38]:
# Change 2019 to 2020 so that it can be joined with 2020 election data. 
crimes_df.loc[crimes_df['YEAR'] == 2019, 'YEAR'] = 2020

# Filter df to include only presidential election year data. 
crimes_df = crimes_df[crimes_df.YEAR.isin(["2000","2004","2008","2012","2016","2020"])].sort_values(by=['YEAR'])
crimes_df

,YEAR,STATE,VIOLENT_CRIME_RATE
0,2000,ALABAMA,21620
28,2000,NEVADA,10474
29,2000,NEW HAMPSHIRE,2167
30,2000,NEW JERSEY,32298
31,2000,NEW MEXICO,13786
...,...,...,...
989,2020,MARYLAND,27456
990,2020,MASSACHUSETTS,22578
991,2020,MICHIGAN,43686
993,2020,MISSISSIPPI,8272


## Guns per Household

In [39]:
# Read in Violent Crime data
url ="https://migration-election-impact.s3.us-east-2.amazonaws.com/Resources/guns_per_household.csv"
spark.sparkContext.addFile(url)
guns_df = spark.read.csv(SparkFiles.get("guns_per_household.csv"), sep=",", header=True, inferSchema=True)
guns_df = guns_df.select("*").toPandas()

In [40]:
# Change column headings to upper case and convert STATE column values to upper case
guns_df = guns_df.rename(columns=str.upper)
guns_df['STATE'] = guns_df['STATE'].str.upper()
guns_df

,YEAR,STATE,GUNS_PER_HOUSEHOLD
0,2000,ALABAMA,0.538
1,2000,ALASKA,0.601
2,2000,ARIZONA,0.427
3,2000,ARKANSAS,0.542
4,2000,CALIFORNIA,0.274
...,...,...,...
845,2016,VIRGINIA,0.353
846,2016,WASHINGTON,0.321
847,2016,WEST VIRGINIA,0.600
848,2016,WISCONSIN,0.471


In [41]:
# Duplicate 2015 values for 2016/2020 and append to df
guns_2020 = guns_df[(guns_df.YEAR ==2016)]

guns_2020.loc[guns_2020['YEAR'] == 2016, 'YEAR'] = 2020

guns_df = guns_df.append(guns_2020,ignore_index=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [42]:
# Filter df to include only presidential election year data. 
guns_df = guns_df[guns_df.YEAR.isin(["2000","2004","2008","2012","2016","2020"])].sort_values(by=['YEAR'])
guns_df


,YEAR,STATE,GUNS_PER_HOUSEHOLD
0,2000,ALABAMA,0.538
27,2000,NEVADA,0.471
28,2000,NEW HAMPSHIRE,0.391
29,2000,NEW JERSEY,0.108
30,2000,NEW MEXICO,0.438
...,...,...,...
869,2020,MARYLAND,0.167
870,2020,MASSACHUSETTS,0.090
871,2020,MICHIGAN,0.389
860,2020,HAWAII,0.091


## GDP_df

In [43]:
# Read in 1997-2020 state GDP data
url ="https://migration-election-impact.s3.us-east-2.amazonaws.com/Resources/State_GDP_97_2020.csv"
spark.sparkContext.addFile(url)
gdp_df = spark.read.csv(SparkFiles.get("State_GDP_97_2020.csv"), sep=",", header=True, inferSchema=True)
gdp_df = gdp_df.select("*").toPandas()

In [44]:
# Change column headings to upper case and convert STATE column values to upper case
gdp_df = gdp_df.rename(columns=str.upper)
gdp_df['STATE'] = gdp_df['STATE'].str.upper()

# Convert YEAR to int 
gdp_df['YEAR'] = gdp_df['YEAR'].astype(int)
gdp_df

,YEAR,STATE,GDP_MILLIONS
0,1997,TEXAS,610141.2
1,1998,TEXAS,645289.5
2,1999,TEXAS,680181.9
3,2000,TEXAS,738804.4
4,2001,TEXAS,772909.2
...,...,...,...
1219,2016,SOUTH DAKOTA,48731.3
1220,2017,SOUTH DAKOTA,50342.9
1221,2018,SOUTH DAKOTA,53239.0
1222,2019,SOUTH DAKOTA,54940.9


In [45]:
# Filter df to include only presidential election year data. 
gdp_df = gdp_df[gdp_df.YEAR.isin(["2000","2004","2008","2012","2016","2020"])].sort_values(by=['YEAR','STATE'])
gdp_df

,YEAR,STATE,GDP_MILLIONS
195,2000,ALABAMA,120132.9
411,2000,ALASKA�,26815.8
267,2000,ARIZONA�,164609.9
867,2000,ARKANSAS�,68740.4
219,2000,CALIFORNIA�,1366166.5
...,...,...,...
935,2020,VIRGINIA,551760.3
119,2020,WASHINGTON,618704.9
647,2020,WEST VIRGINIA,73709.2
695,2020,WISCONSIN�,338678.4


## Combine Dataframes

In [48]:
combined_df = pd.merge(degrees_df, incarceration_df, how='inner', left_on=['YEAR', 'STATE'], right_on = ['YEAR', 'STATE'])
combined_df = pd.merge(combined_df, income_df, how='inner', left_on=['YEAR', 'STATE'], right_on = ['YEAR', 'STATE'])
combined_df = pd.merge(combined_df, crimes_df, how='inner', left_on=['YEAR', 'STATE'], right_on = ['YEAR', 'STATE'])
combined_df = pd.merge(combined_df, gdp_df, how='inner', left_on=['YEAR', 'STATE'], right_on = ['YEAR', 'STATE'])
combined_df = pd.merge(combined_df, guns_df, how='left', left_on=['YEAR', 'STATE'], right_on = ['YEAR', 'STATE'])

In [52]:
demographic_df = combined_df.sort_values(by=['YEAR','STATE'])

In [54]:
# Convert Pandas DF to Spark DF
demographic_df = spark.createDataFrame(demographic_df)
demographic_df

DataFrame[STATE: string, YEAR: bigint, BACHELORS_DEGREE_TOT: bigint, JAIL_RATE: double, PRISON_RATE: double, MED_HOUSE_INC: bigint, VIOLENT_CRIME_RATE: bigint, GDP_MILLIONS: double, GUNS_PER_HOUSEHOLD: double]

In [55]:
# Load clean demographic results into RDS
demographic_df.write.jdbc(url=jdbc_url, table='demographics', mode=mode, properties=config)